In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree, naive_bayes, svm

import sys
sys.path.append('../modules')

from training_realization import Classifier_SVM, Classifier_NB_bin, Classifier_KNN

In [2]:
df = pd.read_csv('../data/bank_additional_preprocessed.csv', sep=';')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 70 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            41188 non-null  int64  
 1   campaign                       41188 non-null  int64  
 2   pdays                          41188 non-null  int64  
 3   emp.var.rate                   41188 non-null  float64
 4   cons.price.idx                 41188 non-null  float64
 5   cons.conf.idx                  41188 non-null  float64
 6   euribor3m                      41188 non-null  float64
 7   nr.employed                    41188 non-null  float64
 8   y                              41188 non-null  int64  
 9   job_admin.                     41188 non-null  int64  
 10  job_blue-collar                41188 non-null  int64  
 11  job_entrepreneur               41188 non-null  int64  
 12  job_housemaid                  41188 non-null 

### Разделим данные на обучение и тест, нормализуем их

In [4]:
X = df.drop('y', axis=1)
y = df.y

Для удобства обработки разделим признаки на категориальные и вещественные

In [5]:
numeric_cols = np.array(['age', 'campaign', 'pdays', 'emp.var.rate', 'cons.price.idx', 
           'cons.conf.idx', 'euribor3m', 'nr.employed'])
X_numeric = X[numeric_cols]

In [6]:
categorical_cols = list(set(X.columns.values.tolist()) - set(numeric_cols))
X_categorical = X[categorical_cols]
for col in categorical_cols:
    X_categorical[col] = X_categorical[col].astype('string')

<ipython-input-6-45ba4a5426bd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_categorical[col] = X_categorical[col].astype('string')


In [7]:
print(X_categorical.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   education_unknown              41188 non-null  string
 1   month_apr                      41188 non-null  string
 2   education_university.degree    41188 non-null  string
 3   marital_unknown                41188 non-null  string
 4   poutcome_nonexistent           41188 non-null  string
 5   marital_single                 41188 non-null  string
 6   job_management                 41188 non-null  string
 7   education_basic.6y             41188 non-null  string
 8   contact_telephone              41188 non-null  string
 9   education_professional.course  41188 non-null  string
 10  education_basic.9y             41188 non-null  string
 11  education_basic.4y             41188 non-null  string
 12  poutcome_failure               41188 non-null  string
 13  m

Разделим данные на обучение и тест

In [8]:
X_train, X_test, X_train_cat, X_test_cat, X_train_num, X_test_num, y_train, y_test = train_test_split(X, X_categorical, X_numeric, y, test_size=0.25)

Нормализируем значения

In [9]:
scaler = StandardScaler()
scaler.fit(X_train_num)

X_train_num_sc = scaler.transform(X_train_num)
X_test_num_sc = scaler.transform(X_test_num)

Соединяем воедино все значения

In [10]:
X_train_transform = np.hstack((X_train_num_sc, X_train_cat))
X_test_transform = np.hstack((X_test_num_sc, X_test_cat))

# Обучение алгоритмов

### KNN

In [11]:
%%time

classifier_KNN = KNeighborsClassifier()
classifier_KNN.fit(X_train_transform, y_train)
y_pred_KNN = classifier_KNN.predict(X_test_transform)
score_KNN = classification_report(y_test, y_pred_KNN)

print(score_KNN)

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      9160
           1       0.53      0.30      0.38      1137

    accuracy                           0.89     10297
   macro avg       0.72      0.63      0.66     10297
weighted avg       0.88      0.89      0.88     10297

Wall time: 17.4 s


### Decision Tree

In [12]:
%%time

classifier_DTC = tree.DecisionTreeClassifier()
classifier_DTC.fit(X_train_transform, y_train)
y_pred_DTC = classifier_DTC.predict(X_test_transform)
score_DTC = classification_report(y_test, y_pred_DTC)

print(score_DTC)

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      9160
           1       0.32      0.35      0.33      1137

    accuracy                           0.85     10297
   macro avg       0.62      0.63      0.62     10297
weighted avg       0.85      0.85      0.85     10297

Wall time: 757 ms


### Naive Bayes

In [13]:
%%time

classifier_NB = naive_bayes.GaussianNB()
classifier_NB.fit(X_train_transform, y_train)
y_pred_NB = classifier_NB.predict(X_test_transform)
score_NB = classification_report(y_test, y_pred_NB)

print(score_NB)

              precision    recall  f1-score   support

           0       0.94      0.83      0.88      9160
           1       0.30      0.57      0.39      1137

    accuracy                           0.81     10297
   macro avg       0.62      0.70      0.64     10297
weighted avg       0.87      0.81      0.83     10297

Wall time: 510 ms


### Support Vector Machine

In [14]:
%%time

classifier_CVM = svm.SVC(kernel='linear')
classifier_CVM.fit(X_train_transform, y_train)
y_pred_CVM = classifier_CVM.predict(X_test_transform)
score_CVM = classification_report(y_test, y_pred_CVM)

print(score_CVM)

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      9160
           1       0.65      0.21      0.31      1137

    accuracy                           0.90     10297
   macro avg       0.78      0.60      0.63     10297
weighted avg       0.88      0.90      0.88     10297

Wall time: 7min 16s


### Logistic Regression

In [15]:
%%time

classifier_LR = LogisticRegression(solver='liblinear', class_weight='balanced')
classifier_LR.fit(X_train_transform, y_train)
y_pred_LR = classifier_LR.predict(X_test_transform)
score_LR = classification_report(y_test, y_pred_LR)

print(score_LR)

              precision    recall  f1-score   support

           0       0.95      0.87      0.90      9160
           1       0.36      0.61      0.45      1137

    accuracy                           0.84     10297
   macro avg       0.65      0.74      0.68     10297
weighted avg       0.88      0.84      0.85     10297

Wall time: 931 ms


# Подбор гиперпараметров

### KNN

In [16]:
%%time

parameters = {'n_neighbors': [1, 2, 3, 4, 5]}

classifier_KNN = KNeighborsClassifier()

clf_KNN = GridSearchCV(classifier_KNN, parameters)
clf_KNN.fit(X_train_transform, y_train)

Wall time: 3min 25s


GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5]})

In [17]:
best_estimator_KNN = clf_KNN.best_estimator_

print(best_estimator_KNN)

KNeighborsClassifier(n_neighbors=4)


### Decision Tree

In [18]:
%%time

parameters = {
    'max_depth': [1, 3, 5, 10, 100],
    'min_samples_split': [2, 3, 5, 8]
}

classifier_DTC = tree.DecisionTreeClassifier()

clf_DTC = GridSearchCV(classifier_DTC, parameters)
clf_DTC.fit(X_train_transform, y_train)

Wall time: 1min 3s


GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [1, 3, 5, 10, 100],
                         'min_samples_split': [2, 3, 5, 8]})

In [19]:
best_estimator_DTC = clf_DTC.best_estimator_

print(best_estimator_DTC)

DecisionTreeClassifier(max_depth=5, min_samples_split=3)


### Naive Bayes

In [20]:
%%time

parameters = {
    'var_smoothing': [1e-10, 1e-9, 1e-8, 1e-5, 1e-2, 1e-1, 1, 10, 100]
}

classifier_NB = naive_bayes.GaussianNB()

clf_NB = GridSearchCV(classifier_NB, parameters)
clf_NB.fit(X_train_transform, y_train)

Wall time: 26.5 s


GridSearchCV(estimator=GaussianNB(),
             param_grid={'var_smoothing': [1e-10, 1e-09, 1e-08, 1e-05, 0.01,
                                           0.1, 1, 10, 100]})

In [21]:
best_estimator_NB = clf_NB.best_estimator_

print(best_estimator_NB)

GaussianNB(var_smoothing=1)


### Support Vector Machine

In [22]:
%%time

parameters = {
    'kernel': ['linear', 'rbf']
}

classifier_CVM = svm.SVC()

clf_CVM = GridSearchCV(classifier_CVM, parameters, cv=3)
clf_CVM.fit(X_train_transform, y_train)

Wall time: 13min 18s


GridSearchCV(cv=3, estimator=SVC(), param_grid={'kernel': ['linear', 'rbf']})

In [23]:
best_estimator_CVM = clf_CVM.best_estimator_

print(best_estimator_CVM)

SVC()


### Logistic Regression

In [24]:
%%time

parameters = {
    'class_weight': ['balanced'],
    'solver': ['liblinear', 'lbfgs', 'newton-cg'],
    'penalty': ['l1', 'l2'],
}

classifier_LR = LogisticRegression()

clf_LR = GridSearchCV(classifier_LR, parameters, cv=3)
clf_LR.fit(X_train_transform, y_train)


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_v

Wall time: 22.5 s


GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid={'class_weight': ['balanced'], 'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'lbfgs', 'newton-cg']})

In [25]:
best_estimator_LR = clf_LR.best_estimator_

print(best_estimator_LR)

LogisticRegression(class_weight='balanced', penalty='l1', solver='liblinear')


### Реализация KNN

Тестирование

In [11]:
cols = list(set(X.columns.values.tolist()))

X_train_transform_int = X_train
for col in cols:
    X_train_transform_int[col] = X_train_transform_int[col].astype('float64')
    
X_test_transform_int = X_test
for col in cols:
    X_test_transform_int[col] = X_test_transform_int[col].astype('float64')


<ipython-input-11-69c41c288556>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_transform_int[col] = X_train_transform_int[col].astype('float64')
<ipython-input-11-69c41c288556>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_transform_int[col] = X_test_transform_int[col].astype('float64')


Так как алгоритм работает медленнее встроенного, вычислим предсказания на 1000 объектах

In [12]:
%%time

user_classifier_KNN = Classifier_KNN()
user_classifier_KNN.fit(X_train_transform_int.values, y_train.values)
y_user_pred_KNN = user_classifier_KNN.predict(X_test_transform_int.values[:1000])

Wall time: 3min 56s


In [13]:
print(list(zip(y_user_pred_KNN[:30], y_test.values[:30])))
score_user_KNN = classification_report(y_test.values[:1000], y_user_pred_KNN)

print(score_user_KNN)

[(0.0, 0), (0.0, 0), (1.0, 0), (1.0, 1), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 1), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 0)]
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       903
           1       0.53      0.29      0.37        97

    accuracy                           0.91      1000
   macro avg       0.73      0.63      0.66      1000
weighted avg       0.89      0.91      0.89      1000



### Реализация Naive Bayes

Тестирование

In [14]:
X_train_transform_int = X_train_cat
for col in categorical_cols:
    X_train_transform_int[col] = X_train_transform_int[col].astype('int64')
    
X_test_transform_int = X_test_cat
for col in categorical_cols:
    X_test_transform_int[col] = X_test_transform_int[col].astype('int64')

<ipython-input-14-e731fd52fcc1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_transform_int[col] = X_train_transform_int[col].astype('int64')
<ipython-input-14-e731fd52fcc1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_transform_int[col] = X_test_transform_int[col].astype('int64')


Значения типа scalar отлично подходят - они равномерно распределены на 0...1

In [15]:
X_train_transform_NB = np.hstack((X_train_num_sc, X_train_transform_int))
X_test_transform_NB = np.hstack((X_test_num_sc, X_test_transform_int))

In [16]:
%%time
print(X_train_transform_NB.shape, y_train.shape)
print(X_test_transform_NB.shape, y_test.shape)
print('Start')

user_classifier_NB = Classifier_NB_bin()
user_classifier_NB.fit(X_train_transform_NB, y_train.values)
y_user_pred_NB = user_classifier_NB.predict(X_test_transform_NB)

print('results shape: ', y_user_pred_NB.shape, y_test.shape)

(30891, 69) (30891,)
(10297, 69) (10297,)
Start
[27357.  3534.]
results shape:  (10297,) (10297,)
Wall time: 3.43 s


In [17]:
score_user_KNN = classification_report(y_test.values, y_user_pred_NB)

print(score_user_KNN)

              precision    recall  f1-score   support

           0       0.94      0.81      0.87      9191
           1       0.27      0.57      0.36      1106

    accuracy                           0.79     10297
   macro avg       0.60      0.69      0.62     10297
weighted avg       0.87      0.79      0.82     10297



### Реализация Support Vector Machines

Тестирование

In [18]:
X_train_transform_SVM = X_train_transform.astype(np.int)
X_test_transform_SVM = X_test_transform.astype(np.int)
y_train_transform_SVM = y_train.values.astype(np.int)
y_test_transform_SVM = y_test.values.astype(np.int)

In [19]:
%%time

user_classifier_SVM = Classifier_SVM(epochs=100)
user_classifier_SVM.fit(X_train_transform_SVM, y_train_transform_SVM)
y_user_pred_SVM = user_classifier_SVM.predict(X_test_transform_SVM)

print('results shape: ', y_user_pred_SVM.shape, y_test.shape)

results shape:  (10297,) (10297,)
Wall time: 35.7 s


In [20]:
user_classifier_SVM = classification_report(y_test.values, y_user_pred_SVM)

print(user_classifier_SVM)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      9191
           1       0.11      1.00      0.19      1106

    accuracy                           0.11     10297
   macro avg       0.05      0.50      0.10     10297
weighted avg       0.01      0.11      0.02     10297



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Добавим побольше штраф за ошибку

In [21]:
%%time

user_classifier_SVM = Classifier_SVM(epochs=200, eta=1e-9, alpha=1e-9)
user_classifier_SVM.fit(X_train_transform_SVM, y_train_transform_SVM)
y_user_pred_SVM = user_classifier_SVM.predict(X_test_transform_SVM)
print(y_user_pred_SVM.shape)
user_classifier_SVM = classification_report(y_test.values, y_user_pred_SVM)

print(user_classifier_SVM)

(10297,)
              precision    recall  f1-score   support

           0       0.89      0.84      0.87      9191
           1       0.11      0.16      0.13      1106

    accuracy                           0.77     10297
   macro avg       0.50      0.50      0.50     10297
weighted avg       0.81      0.77      0.79     10297

Wall time: 1min 13s
